In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Attention, Flatten
from sklearn.metrics import mean_squared_error, mean_absolute_error

2023-12-13 08:58:13.243774: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 08:58:13.288810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 08:58:13.288839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 08:58:13.290275: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 08:58:13.297274: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 08:58:13.298080: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
# Function to load and preprocess the data
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)

     # Assuming datetime is in the first column
    if pd.api.types.is_string_dtype(data.iloc[:, 0]):
        data['date'] = pd.to_datetime(data.iloc[:, 0])
        data['year'] = data['date'].dt.year
        data['month'] = data['date'].dt.month
        data['day'] = data['date'].dt.day
        data['hour'] = data['time'].dt.hour
        data['minute'] = data['time'].dt.minute
        data['second'] = data['time'].dt.second
        data.drop(columns=['date', data.columns[0]], inplace=True)
    
    # Handle missing values (if any)
    data.fillna(method='ffill', inplace=True)
    data.fillna(method='bfill', inplace=True)

    # Normalize the data
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data.values)

    # Assuming the last column is the target
    X = data_scaled[:, :-1]
    y = data_scaled[:, -1]

    # Reshape X for LSTM [samples, time steps, features]
    X = np.expand_dims(X, axis=1)

    return X, y

In [5]:
# Load the datasets
train1_X, train1_y = load_and_preprocess_data('/home/disi/ff/Datasets/Dataset_2/Train1.csv')
train2_X, train2_y = load_and_preprocess_data('/home/disi/ff/Datasets/Dataset_2/Train2.csv')
test_X, test_y = load_and_preprocess_data('/home/disi/ff/Datasets/Dataset_2/Test.csv')

# Combine training data
train_X = np.concatenate([train1_X, train2_X], axis=0)
train_y = np.concatenate([train1_y, train2_y], axis=0)

/tmp/ipykernel_15693/741495675.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_15693/741495675.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


ValueError: could not convert string to float: '2015-02-04 17:51:00'